# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# импорт библиотеки pandas
import pandas as pd

In [2]:
# чтение файла с данными и сохранение в переменной data
data = pd.read_csv('/datasets/data.csv')
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# получение общей информации о данных в таблице data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. 

Согласно документации к данным:

* children — количество детей в семье;
* days_employed — общий трудовой стаж в днях;
* dob_years — возраст клиента в годах;
* education — уровень образования клиента;
* education_id — идентификатор уровня образования;
* family_status — семейное положение;
* family_status_id — идентификатор семейного положения;
* gender — пол клиента;
* income_type — тип занятости;
* debt — имел ли задолженность по возврату кредитов;
* total_income — ежемесячный доход;
* purpose — цель получения кредита.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

Так же в столбце days_employed некорректные данные: во-первых тип данных  вещественное число, а во-вторых числа имеют отрицательные значения. 

In [4]:
#поиск пропусков методом isna()
data[data['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [5]:
# определение количества пропусков
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Вывод**

1. В таблице с данными есть пропуски в столбцах days_employed и total_income. Количество пропусков - 2174 в каждом столбце. Столбцы, посвященны данным о работе клиентов, их отсутствие может говорить о том, что клиент не подавал данные о работе при взятии кредита и брал кредит под более большой процент из-за этого. Пропуски составляют 10,1% от общего количества, это достаточно много и при удалении строк с ними приведет к менее достоверному исследованию. К тому же нас интересуют данные о семейном положении и наличии детей для иследования, а они в этих данных не пропущены. Поэтому удалять строки не будем, а заменим значение NaN в уровне дохода и стаже на медиану. 

2. Так же в столбце days_employed есть сильно завышенное значение - 340266.072047, что соответствует 932 года стажа, что точно не реально, поэтому заменим такие большие значения на медиану. Некоторые значения сильно выделяются среди большинства, поэтому и берем медиану, а не среднее значение. 


In [6]:
# найдем уникальные значения и их количество в столбце 'dob_years' 
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [7]:
# найдем уникальные значения и их количество в столбце 'gender' 
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [8]:
# найдем уникальные значения и их количество в столбце 'children' 
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


**Вывод**

1. В столбце dob_years 101 раз повторяется 0 в возрасте клиентов, это всего лишь 0,47% от общего числа, но удалять не будем, так как данные о семье и детях есть в этих строках, просто заменим 0 на медиану по возрасту. Заменим на медиану, а не на средний возраст, с целью сохрания целых чисел в данном столбце.

2. В столбце gender 1 раз встречается непонятный символ XNA, который можно расценить как пропуск. В иследовании не важен пол, поэтому можно заменить на 'unknown'.

3. В столбце children, встречается значения -1 ребенок - 47 раз, возможно это опечатка. А вот количество 20 детей это конечно здорово, но в наше время почти не реально. Я нашла только одну семью в России, имеющих столько детей. Возможно есть еще, которых не выдал яндекс, но 76 семей, имеющее такое количество детей, это похоже на маловероятный факт. Природу ошибки мы узнать не можем и так как данных значений меньше 1%, просто удалим их.

In [9]:
#найдем с помощью метода duplicated(), количество дубликатов в таблице с данными
data.duplicated().sum()

54

In [10]:
#выведем на экран таблицу с дубликатами для ознакомления
data_duplicated = data[data.duplicated()].head() 
data_duplicated

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,покупка жилья для семьи
4182,1,NaN,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,NaN,свадьба
4851,0,NaN,60,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,свадьба
5557,0,NaN,58,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,сыграть свадьбу
7808,0,NaN,57,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,на проведение свадьбы


In [11]:
#выведем все уникальные значения столбца 'education'
data['education'].unique() 

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [12]:
#выведем все уникальные значения столбца 'purpose'
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Вывод**

С помощью метода duplicated(), выявили явные дубликаты в таблице - 54. 

Много неявных дубликатов, обнаружила благодаря методу unique(). Возможно цель для получения кредита клиенты не выбирали готовые варианты, а заполняли сами "от руки" свой вариант, поэтому они так разнятся.

Можно объеденить ряд похожих целей в столбце purpose в четыре основных:
* недвижимость;
* автомобиль;
* образование;
* свадьба;


А в столбце education, привести все к нижнему регистру.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [13]:
#заполним отсутствующие значения столбцов 'days_employed' и 'total_income' медианой при помощи цикла.
columns_to_replace = ['days_employed', 'total_income']
for column in columns_to_replace:
    data[column] = data[column].fillna(data[column].median()) 

In [14]:
# убедимся, что в таблице не осталось пропусков.
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [15]:
# заменим ноль в столбце dob_years, на медиану по возрасту
data['dob_years'] = data['dob_years'].replace(0, data['dob_years'].median()) 
#data['dob_years'].value_counts()

In [16]:
# заменим XNA в столбце gender на 'unknown'
data['gender'] = data['gender'].replace('XNA', 'unknown') 
data['gender'].value_counts()

F          14236
M           7288
unknown        1
Name: gender, dtype: int64

In [17]:
# удалим строки со значениями -1 и 20 с столбце children
data = (data.drop(index=data.query('children == -1').index))
data = (data.drop(index=data.query('children == 20').index))

In [18]:
# проверим результат
data['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**

Для того, чтобы максимально сохранить данные, после выявления пропусков и подозрительных значений, для их замены мы применили методы fillna, replace и логическую индексацию.

### Замена типа данных

In [19]:
# для удобства просмотра данных меняем тип данных на целочисленный в столбцах days_employed и total_income
data[['days_employed', 'total_income']] = data[['days_employed', 'total_income']].abs().astype('int')
data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10403,2,62,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,100452,профильное образование
12752,0,289,42.0,высшее,0,Не женат / не замужем,4,M,компаньон,0,191940,недвижимость
19664,0,1227,58.0,среднее,1,гражданский брак,1,M,сотрудник,1,100045,свадьба
13170,0,1203,55.0,среднее,1,женат / замужем,0,F,госслужащий,0,145017,высшее образование
7573,0,1203,62.0,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы


In [20]:
#убедимся, что данные изменились
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  int64  
 2   dob_years         21402 non-null  float64
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  int64  
 11  purpose           21402 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


In [21]:
# заменим в колонке days_employed значения, которые превышают средний стаж работы по России, а это примерно 35 лет (35*365=12775) 
# меняем на медиану значений в данной колонке
data.loc[data['days_employed'] >= 12775, 'days_employed'] = int(data['days_employed'].median())      

In [22]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,1810,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Для удобства просмотра, мы заменили значения в столбцах days_employed и total_income, на целочисленные методом astype() и убрали отрицательные значения методом abs() в столбце days_employed. Так же заменили стаж работы, который превышал возраст клиента, на медиану значений в данном столбце.

### Обработка дубликатов

In [23]:
# удалим явные дубликаты при помощи метода drop_duplicates() и проверим все ли дубликаты удалились.
data = data.drop_duplicates().reset_index(drop=True) 
data.duplicated().sum()

0

In [24]:
# приведем столбец 'education' к нижнему регистру
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**Вывод**

Мы избавились от явных дубликатов при помощи метода drop_duplicates(). 

Привели столбец education с данными об образовании к нижнему регистру для удобства операций с ним.

### Лемматизация

In [25]:
#загружаем библиотеку 
from pymystem3 import Mystem
m = Mystem()
#создадим функцию, которая пробежится по всем значениям столбца 'purpose', 
#эти значения разбиваем на категории по леммам: "жилье/недвижимость", "автомобиль", "образование", "свадьба".
#присваеваем каждой основе слова, свою категорию.
def purpose_row(purpose):
    lemmas = ' '.join(m.lemmatize(purpose))
    if 'авто' in lemmas:
        return 'автомобиль'
    
    if 'недвиж' in lemmas or 'жил' in lemmas:
        return  'недвижимость'
    
    if 'свад' in lemmas:
        return  'свадьба'
    
    if 'образов' in lemmas:
        return 'образование' 
#присоединяем к таблице новый столбец с объединенными категориями
data['purpose_group'] = data['purpose'].apply(purpose_row)        
data.sample(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
8050,0,1810,62.0,среднее,1,женат / замужем,0,F,пенсионер,0,196595,покупка жилья для семьи,недвижимость
9816,2,2448,38.0,среднее,1,женат / замужем,0,M,сотрудник,0,168148,заняться высшим образованием,образование
20609,0,1810,58.0,среднее,1,в разводе,3,F,пенсионер,0,180094,автомобили,автомобиль
5439,2,2229,40.0,высшее,0,женат / замужем,0,F,госслужащий,0,162199,покупка недвижимости,недвижимость
19241,1,2424,45.0,высшее,0,женат / замужем,0,F,компаньон,0,197454,на покупку своего автомобиля,автомобиль
6552,1,1553,43.0,среднее,1,женат / замужем,0,M,сотрудник,0,115092,ремонт жилью,недвижимость
7156,0,3903,28.0,высшее,0,гражданский брак,1,F,сотрудник,0,211662,строительство собственной недвижимости,недвижимость
5732,0,1810,62.0,среднее,1,женат / замужем,0,F,пенсионер,0,76004,покупка жилья для семьи,недвижимость
4814,0,1810,65.0,среднее,1,женат / замужем,0,F,пенсионер,0,190439,на покупку автомобиля,автомобиль
14788,2,3399,31.0,среднее,1,женат / замужем,0,F,сотрудник,0,186694,получение высшего образования,образование


**Вывод**

С помощью лемматизации мы создали отделный столбец, в котором выделили 4 основыных целей для кредита. 

### Категоризация данных

In [26]:
# создадим функцию, для того, чтобы категоризировать клиентов на две группы: "дети есть" и "детей нет"
def children_group(row):

    children = row['children']

    if children >= 1:
        return 'дети есть'

    if children == 0:
        return 'детей нет'
    
data['children_group'] = data.apply(children_group, axis=1)
data.sample(5) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,children_group
950,0,727,33.0,среднее,1,Не женат / не замужем,4,F,компаньон,0,233913,жилье,недвижимость,детей нет
18266,0,197,40.0,высшее,0,в разводе,3,M,сотрудник,0,157729,недвижимость,недвижимость,детей нет
4567,0,7927,45.0,среднее,1,Не женат / не замужем,4,F,компаньон,0,91240,недвижимость,недвижимость,детей нет
12736,0,495,29.0,среднее,1,Не женат / не замужем,4,M,сотрудник,1,135737,операции с жильем,недвижимость,детей нет
4627,2,2283,38.0,среднее,1,женат / замужем,0,F,сотрудник,0,72774,автомобили,автомобиль,дети есть


In [27]:
# создадим функцию, для того чтобы категоризировать клиентов по уровню дохода.
# граница дохода рассматривалась, по субъективному мнению.
# добавим столбец с группировкой по доходу.
def total_income_group(row):

    total_income = row['total_income']

    if total_income > 300000:
        return 'высокий'

    if 100000 < total_income <300000:
        return 'средний'
    
    if total_income < 100000:
        return 'низкий'
    
data['total_income_group'] = data.apply(total_income_group, axis=1)
data.head() 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,children_group,total_income_group
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,дети есть,средний
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,дети есть,средний
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,детей нет,средний
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,дети есть,средний
4,0,1810,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,детей нет,средний


**Вывод**

Для удобства сравнения при дальнейших ответах на вопросы, сделали категоризацию по наличию детей и уровню дохода и добавили к таблице с данными data, соответствующие столбцы. 

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

0 - задолжность есть

1 - задолжности нет

In [28]:
# сгрупируем по колонке 'children_group' и посчитаем сумму по колонке 'debt'
data.groupby('children_group')['debt'].sum()

children_group
детей нет    1063
дети есть     669
Name: debt, dtype: int64

**Промежуточный вывод**

Видно, что клиенты, у которых детей нет, реже допускают просрок по кредиту, чем люди с детьми.

Посмотрим более детально, как количество детей влияет на количество просроченный кредитов.

In [29]:
# создадим таблицу, сгрупированную по количеству детей, где в столбцах посчитаем 
# количество и сумму задолжности по кредиту. Добавим столбец с конверсией по задолжности.
data_children = pd.DataFrame()
data_children['count_children'] = data.groupby('children')['debt'].count()
data_children['sum_children'] = data.groupby('children')['debt'].sum()
data_children['result_children'] = data_children['sum_children'] / data_children['count_children'] 
data_children.sort_values('result_children', ascending = False)

,count_children,sum_children,result_children
children,,,
4,41,4,0.097561
2,2052,194,0.094542
1,4809,444,0.092327
3,330,27,0.081818
0,14106,1063,0.075358
5,9,0,0.000000


**Вывод**

Видно, что с увеличением количества детей, увеличивается количество просроченных кредитов. 

Клиенты с пятью детьми, ниразу не выплатили кредит в срок.

Больше всего берут кредиты, клиенты без детей. возврат по кредиту у них выше всех.

Клиенты с тремя детьми, возвращают кредиты чаще, чем с одним ребенком.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
# создадим таблицу, сгрупированную по семейному статусу, где в столбцах посчитаем 
# количество и сумму задолжности по кредиту. Добавим столбец с конверсией по задолжности.
data_family_status = pd.DataFrame()
data_family_status['count_family_status'] = data.groupby('family_status')['debt'].count()
data_family_status['sum_family_status'] = data.groupby('family_status')['debt'].sum()
data_family_status['result_family_status'] = data_family_status['sum_family_status'] / data_family_status['count_family_status'] 
data_family_status.sort_values('result_family_status', ascending = False)

,count_family_status,sum_family_status,result_family_status
family_status,,,
Не женат / не замужем,2796,273,0.097639
гражданский брак,4145,385,0.092883
женат / замужем,12266,927,0.075575
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246


**Вывод**

Всех больше невозвратов по кредиту у не женатых/не замужних. Самые ответственные плательщики по кредиту - это вдовы/вдовцы. Одиночки, но которые после развода платят кредиты более ответственно, нежели одиночки, которые не были женаты/замужем.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [31]:
# создадим таблицу, сгрупированную по уровню дохода, где в столбцах посчитаем 
# количество и сумму задолжности по кредиту. Добавим столбец с конверсией по задолжности.
data_total_income = pd.DataFrame()
data_total_income['count_total_income'] = data.groupby('total_income_group')['debt'].count()
data_total_income['sum_total_income'] = data.groupby('total_income_group')['debt'].sum()
data_total_income['result_total_income'] = data_total_income['sum_total_income'] / data_total_income['count_total_income'] 
data_total_income.sort_values('result_total_income', ascending = False)

,count_total_income,sum_total_income,result_total_income
total_income_group,,,
средний,15430,1272,0.082437
низкий,4444,354,0.079658
высокий,1473,106,0.071962


**Вывод**

Мы видим, что клиенты с высоким уровнем дохода, берут кредиты почти в 10 раз меньше, чем клиенты со средним уровнем дохода. Зависимость между уровнем дохода и возвратом кредита есть. Люди с высоким уровнем дохода меньше допускают просроков по кредиту. Клиенты со средним уровнем дохода, имеют задолжность по кредиту чаше, чем люди с низким и высоким уровнем дохода.

Но полученный результат нельзя считать достоверным, так как в столбце с уровнем дохода, было пропущено много значений и замена медианой, не гарантирует точный результат. Так же при категоризации по доходу, мы взяли границы по субъективному мнению, что тоже может исказить результат. 

- Как разные цели кредита влияют на его возврат в срок?

In [32]:
# создадим таблицу, сгрупированную по цели кредита, где в столбцах посчитаем 
# количество и сумму задолжности по кредиту. Добавим столбец с конверсией по задолжности.
data_purpose_group = pd.DataFrame()
data_purpose_group['count_purpose_group'] = data.groupby('purpose_group')['debt'].count()
data_purpose_group['sum_purpose_group'] = data.groupby('purpose_group')['debt'].sum()
data_purpose_group['result_purpose_group'] = data_purpose_group['sum_purpose_group'] / data_purpose_group['count_purpose_group'] 
data_purpose_group.sort_values('result_purpose_group', ascending = False)

,count_purpose_group,sum_purpose_group,result_purpose_group
purpose_group,,,
автомобиль,4281,400,0.093436
образование,3989,369,0.092504
свадьба,2323,183,0.078777
недвижимость,10754,780,0.072531


**Вывод**

Мы видим, что задолжности по кредиту меньше у людей, которые берут деньги на недвижимость и свадьбу. Возможно, это связанно с тем, что когда люди берут деньги на недвижимость, они все детальней просчитывают и более ответственно подходят к делу, чтобы не потерять "крышу над головой". А на свадьбу сумма кредита не очень велика, что ее не так сложно вернуть.

## Шаг 4. Общий вывод

I. В предобрабтке данных:
 * Мы нашли пропуски - явные и неявные, проанализировали возможные причины их появления  и заполнили их. 
 * Нашли аномальные значения и тоже их устранили.
 * Привели столбцы 'days_employed' и 'total_income' к целочисленному типу данных и заменили все отрицательные значения столбца 'days_employed' на положительные.
 * Так же мы выявили все дубликаты - явные и неявные и проработали их. 
 * В столбце 'purpose' мы обнаружили, что похожие цели написаны в разных вариациях. И при помощи лемматизации объеденили ряд похожих целей в четыре основных: недвижимость, автомобиль, образование, свадьба.
 * Далее для удобства ответов на вопросы, мы добавили к таблице с данными два столбца, в которых категоризировали данные. В первом столбце поделили клиентов на имеющих детей и неимеющих. Во втором, по уровню дохода.
   

II. Мы проверили четыре гипотезы и установили:

  * Да зависимость между наличием детей и возвратом кредита в срок есть. Клиенты не имеющие детей реже допускают просрок по кредиту.

  * Есть зависимость между семейным положением и возвратом кредита в срок. Всех больше невозвратов по кредиту у не женатых/не замужних. Самые ответственные плательщики по кредиту - это вдовы/вдовцы. 

  * Зависимость, между уровнем дохода и возвратом кредита в срок есть, но ее нельзя считать достоверной, так как было пропущено 10,1% данных и замена пропусков медианой, не гарантирует достоверный результат. Так же разница между конверсиями не велика, что в действительности может иметь противоположный результат, так как категоризация по уровню дохода была взята условно.

  * Зависимость, как цели кредита влияют на его возврат в срок - есть. Мы видим, что задолжности по кредиту меньше у людей, которые берут деньги на недвижимость и свадьбу.

III. Рекомендации:

 * Можно порекомендовать банку откорретировать анкету "заявка на получения кредита", в которой, в графах "цели кредита" и "образования", предложить потенциальным заёмщикам выбирать из готовых вариантов ответов, тем самым мы уменьшим количество неявных дубликатов и упростим дальнейший анализ данных.

 * Конечно из данных одного банка не всегда можно сделать вывод о всех жителях города, но все же для оценки способности потенциального заёмщика вернуть кредит банку можно учитывать при построении модели **кредитного скоринга** семейное положение и количество детей, а так же цели для взятия кредита. Потому что после проверки гипотез, мы проследили, что есть зависимость между этими вопросами и возвратом кредита в срок.

